In [86]:
import pip
from bs4 import BeautifulSoup
from pip._vendor import requests

import re
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    text = TAG_RE.sub('', text)
    return text

def loadLexicon(wordFile):
    """Returns a set of words from a file"""
    # return set(line.strip() for line in open(wordFile))
    return list(set(line.strip() for line in open(wordFile)))

POS_FILE, NEG_FILE = 'positive-words.txt', 'negative-words.txt'
POS_LEX, NEG_LEX = loadLexicon(POS_FILE), loadLexicon(NEG_FILE)
# line='“well done! precise implementation of the design specification and very fast response time”'
def comp(list1, list2):
    list3 = []
    for val in list1:
        if val in list2:
            list3.append(val)
    return list3

def get_sentiment(line, posLex=POS_LEX, negLex=NEG_LEX):
    """Gets sentiment for a line based on the lexicons.
    returns line, sentiment, positive_words, and negative_words
    """
    # words = set(line.split(' '))
    words = line.split(' ')
    # posList = words & posLex  # set intersection
    # negList = words & negLex
    posList = comp(words, posLex)
    negList = comp(words, negLex)
    diff = len(posList) - len(negList)
    sentiment = 'Neutral' if diff == 0 else 'Positive' if diff > 0 else 'Negative'
    # return (sentiment, posList, negList)
    return (sentiment, posList, negList, diff)


userFL=[]      # Name
rateFL=[]      # rate per hour
ratingsFL=[]      # ratings
cityFL = []      # city
countryFL = []      # country
latList = [];      # list of latitudes
lonList = [];      # list of longitudes

#get the names of 200 users with skill = all from 20 pages (10 per page)
for i in range(1, 21):
    temp = 'https://www.freelancer.com/freelancers/skills/all/' + str(i)
    content = requests.get(temp).content
    soup = BeautifulSoup(content, 'html.parser')
    userFL += [element.get_text() for element in soup.select('a.find-freelancer-username')]
    userFL = list(map(str.strip, userFL))

print('Retrieving records..')
print('')

import texttable as tt
for j in range(0, 200):
    temp1 = 'https://www.freelancer.com/u/' + userFL[j] + '.html'
    content = requests.get(temp1).content
    soup = BeautifulSoup(content, 'html.parser')
    cityFL += [element.get_text() for element in soup.select('span.PageProfile-info-locality')]
    countryFL += [element.get_text() for element in soup.select('span.profile-location-name')]
    rateFL += [element.get_text() for element in soup.select('span.hourly-rate-value')]
    ratingsFL += [element.get_text() for element in soup.select('span.Rating-review')]
    jtemp = j+1
    review = soup.find_all('p', {'itemprop': 'reviewBody'})
    print('Name: '+str(userFL[j]).strip())
    print('City: '+str(cityFL[j]).strip())
    print('Country: '+str(countryFL[j]).strip())
    print('Rate: '+str(rateFL[j]).strip())
    print('Ratings: '+str(ratingsFL[j]).strip())
    count=0
    print('Reviews: ')

    tab = tt.Texttable()
    rowx = [[]] # The empty row will have the header    
    count = 1
    pos_cnt=0
    neg_cnt=0
    for x in review:
        fin_text = remove_tags(str(x))
        fin_text1 = fin_text[1:-1]
        fin_text1= fin_text1.replace("."," ")
        fin_text1= fin_text1.replace(","," ")
        fin_text1= fin_text1.replace("!"," ")
        fin_text1= fin_text1.lower()
        tmp = get_sentiment(fin_text1)
        rowx.append([count,fin_text,tmp[0]])
        count=count+1
        if tmp[0]=='Positive':
            pos_cnt = pos_cnt+1
        else:
            neg_cnt = neg_cnt+1
    tab.add_rows(rowx)
    tab.set_cols_align(['l','l','l'])
    tab.set_cols_width([9,60,9])
    tab.header(['Review No', 'Review', 'Sentiment'])
    print(tab.draw())
    
    if pos_cnt>neg_cnt:
        print('User\'s overall review: Positive')
    elif pos_cnt<neg_cnt:
        print('Users overall review: Negative')
    else:
        print('Users overall review: Neutral')
    print('')
    print('')
# remove the white spaces
rateFL = list(map(str.strip, rateFL))
ratingsFL = list(map(str.strip, ratingsFL))
cityFL = list(map(str.strip, cityFL))
countryFL = list(map(str.strip, countryFL))

print('')

# !pip install geopy 
# Intsall the geopy package if not installed 
from geopy.geocoders import Nominatim
geolocator = Nominatim()
    
for j in range(0, 200):
    ss=str(cityFL[j])
    location = geolocator.geocode(ss)
    if location is None: #If the coordinates of the city are not present, then make the country, the location
        ss=str(countryFL[j])
        location = geolocator.geocode(ss)  
    latList += [location.latitude]
    lonList += [location.longitude]
    jtemp = j+1
    print('Retrieving coordinates '+ str(jtemp))

import folium
import numpy as np

from folium.plugins import MarkerCluster
locations = list(zip(latList,lonList))

popups = rateFL # rate per hour
# ['{}'.format(a) for a in temp]

mapa = folium.Map(location=[np.mean(latList), np.mean(lonList)],
                  tiles='Cartodb Positron', zoom_start=1)

mapa.add_child(MarkerCluster(locations=locations, popups=popups))
print('')
print('Plotting the map...')
mapa

Retrieving records..

Name: mikehurley
City: jaipur
Country: India
Rate: $12
Ratings: 2618 reviews
Reviews: 
+-----------+--------------------------------------------------------------+-----------+
| Review No |                            Review                            | Sentiment |
+===========+==============================================================+===========+
| 1         | “Good job,  thanks!”                                         | Positive  |
+-----------+--------------------------------------------------------------+-----------+
| 2         | “Thanks for work on ongoing project.”                        | Positive  |
+-----------+--------------------------------------------------------------+-----------+
| 3         | “Been utilizing this Freelancer for a long time and very     | Positive  |
|           | satisfied.”                                                  |           |
+-----------+--------------------------------------------------------------+-----------+
|

In [85]:
!pip install texttable 
!pip install geopy 
#Run this cell first if you haven't installed the above mentioned modules